# Write your first Qiskit Serverless program

This example demonstrates how to create a parallel transpilation program and deploy it to IBM Quantum&trade; Platform to use as a reusable remote service.

## Example: remote transpilation with Qiskit Serverless

Start with the following example that transpiles a `circuit` against a given `backend` and target `optimization_level`, and gradually add more elements to deploy your workload to Qiskit Serverless.

In [2]:
# /source_files/transpile_remote.py

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

def transpile_remote(circuit, optimization_level, backend):
    """Transpiles an abstract circuit into an ISA circuit for a given backend."""
    pass_manager = generate_preset_pass_manager(
        optimization_level=optimization_level,
		backend=backend
    )
    isa_circuit = pass_manager.run(circuit)
    return isa_circuit

## Set up your files

Serverless requires setting up your workload’s `.py` files into a dedicated directory. The following structure is an example of good practice:

```text
serverless_program
|___ program_uploader.ipynb
|___ source_files
     |___ transpile_remote.py
     |___ *.py
```

Serverless uploads the contents of `source_files` to run remotely. Once these are set up, you can adjust `transpile_remote.py` to fetch inputs and return outputs. 

### Get program arguments

Your initial `transpile_parallel.py` has two inputs: `circuits` and `backend`. Serverless is currently limited to only accept serializable inputs and outputs. For this reason, you cannot pass in `backend` directly, so use `backend_name` as a string instead.

In [ ]:
# /source_files/transpile_remote.py

from qiskit_serverless import get_arguments, save_result, distribute_task, get

# Get program arguments
arguments = get_arguments()
circuits = arguments.get("circuits")
backend_name = arguments.get("backend_name")
optimization_level = arguments.get("optimization_level")

At this point, you can get your backend with `QiskitRuntimeService`, and add your existing program:

In [ ]:
# /source_files/transpile_remote.py

service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.get_backend(backend_name)

Finally, we can setup running `transpile_remote()` across all `circuits` passed in, and return the `transpiled_circuits` as a result:

In [ ]:
# /source_files/transpile_remote.py

results = [
    transpile_remote(circuit, backend)
    for circuit in circuits
]

save_result({
    "transpiled_circuits": results
})

## Deploy to IBM Quantum Platform

Authenticate to the `IBMServerlessClient` with your IBM Quantum token, which you can find in your [IBM Quantum account](https://quantum.ibm.com/account), and upload the program.

In [2]:
from qiskit_serverless import IBMServerlessClient, QiskitFunction

# Authenticate to the remote cluster and submit the pattern for remote execution
serverless = IBMServerlessClient(
    token=API_TOKEN
)

Qiskit Serverless compresses the contents of `working_dir` (in this case, `source_files`) into a `tar`, which gets uploaded and cleaned up after. The `entrypoint` identifies the main program executable for Qiskit Serverless to run. Additionally, if your program has custom `pip` dependencies, you can add them to a `dependencies` array:

In [3]:
transpile_remote_demo = QiskitFunction(
    title="transpile_remote_serverless",
    entrypoint="transpile_remote.py",
    working_dir="./source_files/",
	dependencies=["qiskit-aer==0.14.1"],
)

In [4]:
serverless.upload(transpile_remote_demo)

'transpile_remote_serverless'

To check if it successfully uploaded, use `serverless.list()`:

In [5]:
serverless.list()

[QiskitFunction(transpile_remote_serverless)]

In the next section, we’ll explore how to pass inputs and run this program remotely. 